# Discovery of Frequent Itemsets and Association Rules

1. Frequent itemsets with support s
2. Association Rules with confidence c


A priori:
C1
    - read transactions and get counts
L1
    - filter candidates with support s

CK
    - generate candidates CK out of L(k-1), with monotonicity support s,
    - read transactions and get counts
LK
- filter candidates LK with support s



Keep in mind
- if the files are very large and can't be fit into memory, then these files should be split. With a dataloader

In [123]:
def load_transactions(filepath):
    transactions = []
    
    with open(filepath, 'r') as f:
        for line in f:
            # Convert each line to a set of integers
            transaction = set(map(int, line.strip().split()))
            transactions.append(transaction)
    
    return transactions

transactions = load_transactions('data/T10I4D100K.dat')
print(f"Number of transactions: {len(transactions)}")
print(f"First few transactions: {transactions[:3]}")

all_items = set().union(*transactions)
print(f"Total unique items: {len(all_items)}")

Number of transactions: 100000
First few transactions: [{448, 834, 164, 775, 328, 687, 240, 368, 274, 561, 52, 630, 825, 25, 538, 730}, {704, 834, 581, 39, 205, 814, 401, 120, 825, 124}, {674, 35, 712, 854, 759, 950, 249, 733}]
Total unique items: 870


# A Priori algorithm

the idea is to incrementally build larger itemsets by combining frequently occuring smaller subsets.


### Frequent singletons:

    1. find singletons
    2. c1 <- count singletons
    3. l1 <- filter frequent c1

### Frequent 2-itemsets

    1. candidates <- combine singletons to 2-itemsets
    2. c2 <- count candidates
    3. l2 <- filter c2

### Frequent k-itemsets
    1. candidates <- combine singletons with k-1 item sets
    2. ensure frequent subsets - they occur in all frequently filtered candidates L (consisting of each lk)
    3. ck <- count candidates
    4. lk <- filter ck



    




In [124]:
from tqdm import tqdm

def A_priori(transactions, s, max_k=3):
    # get counts for singletons (k=1)
    c1 = {frozenset([item]): 0 for item in all_items}
    for transaction in transactions:
        for item in transaction:
            c1[frozenset([item])] += 1
    
    # filter singletons with support s  
    l1 = {itemset: count for itemset, count in c1.items() if count >= s}
    
    # initialize result with L1
    L = [l1]
    C = [c1]
    
    print(f"C1: {len(c1)} - {list(c1)[:5]}..")
    print(f"L1: {len(l1)} - {list(l1)[:5]}..")
    
    # iterate for k=2 to max_k
    for k in range(2, max_k + 1):
        # generate candidates from previous frequent itemsets
        ck = {}
        prev_l = L[k-2]
        
        # generate candidates by combining previous frequent itemsets
        for itemset1 in prev_l:
            for singleton in l1:
                union = itemset1.union(singleton)
                if len(union) == k:
                    
                    # subset frequency check
                    all_subsets_frequent = True
                    for item in union:
                        subset = frozenset(union - {item})
                        if subset not in prev_l:
                            all_subsets_frequent = False
                            break
                    
                    if all_subsets_frequent:
                        ck[union] = 0
        
        print(f"C{k} candidates: {len(ck)} - {list(ck)[:5]}..")
 
        # count occurrences of candidates
        for transaction in tqdm(transactions):
            transaction_set = frozenset(transaction)
            for candidate in ck:
                if candidate.issubset(transaction_set):
                    ck[candidate] += 1
        
        print(f"C{k} counts: {len(ck)} - {list(ck)[:5]}..")
        C.append(ck)
        
        # filter candidates with minimum support
        lk = {itemset: count for itemset, count in ck.items() if count >= s}
        
        print(f"L{k}: {len(lk)} - {lk}")
        
        # if no frequent itemsets found, break
        if not lk:
            break
        L.append(lk)
            
    
    return L, C



In [125]:
s = 0.05*len(transactions)
print(f"s: {s}")
L, C = A_priori(transactions, s, 3)


s: 5000.0
C1: 870 - [frozenset({0}), frozenset({1}), frozenset({2}), frozenset({3}), frozenset({4})]..
L1: 10 - [frozenset({217}), frozenset({354}), frozenset({368}), frozenset({419}), frozenset({494})]..
C2 candidates: 45 - [frozenset({217, 354}), frozenset({368, 217}), frozenset({217, 419}), frozenset({217, 494}), frozenset({217, 529})]..


100%|██████████| 100000/100000 [00:00<00:00, 605564.61it/s]

C2 counts: 45 - [frozenset({217, 354}), frozenset({368, 217}), frozenset({217, 419}), frozenset({217, 494}), frozenset({217, 529})]..
L2: 0 - {}


In [126]:
L

[{frozenset({217}): 5375,
  frozenset({354}): 5835,
  frozenset({368}): 7828,
  frozenset({419}): 5057,
  frozenset({494}): 5102,
  frozenset({529}): 7057,
  frozenset({684}): 5408,
  frozenset({722}): 5845,
  frozenset({766}): 6265,
  frozenset({829}): 6810}]

In [127]:
c2 = sorted([(v,c) for v,c in C[1].items()], key=lambda x: x[1], reverse=True)
print(c2)


[(frozenset({368, 829}), 1194), (frozenset({368, 494}), 860), (frozenset({368, 529}), 640), (frozenset({684, 766}), 613), (frozenset({529, 829}), 584), (frozenset({722, 354}), 566), (frozenset({368, 766}), 504), (frozenset({217, 722}), 498), (frozenset({722, 684}), 443), (frozenset({217, 529}), 403), (frozenset({368, 722}), 392), (frozenset({368, 684}), 387), (frozenset({722, 419}), 366), (frozenset({368, 419}), 355), (frozenset({684, 829}), 349), (frozenset({217, 419}), 344), (frozenset({529, 684}), 334), (frozenset({354, 766}), 329), (frozenset({722, 766}), 328), (frozenset({829, 766}), 321), (frozenset({368, 354}), 319), (frozenset({529, 766}), 317), (frozenset({368, 217}), 303), (frozenset({529, 354}), 301), (frozenset({722, 829}), 294), (frozenset({529, 722}), 283), (frozenset({217, 354}), 280), (frozenset({217, 766}), 276), (frozenset({217, 829}), 275), (frozenset({829, 494}), 267), (frozenset({354, 419}), 263), (frozenset({354, 829}), 259), (frozenset({419, 829}), 259), (frozens

# Association Rules

1. frequent itemset I
2. rule generation
    - for subset A in I:
    
        1. conf(A -> A/I) = supp(I) / supp(A)              given A how likely to also observe A/I

        2. if conf > c:

            We've found a rule!

In [128]:
transactions = [
    {1, 2, 3},    # B1 {m, c, b}
    {1, 4, 5},    # B2 {m, p, j}
    {1, 2, 3, 6}, # B3 {m, c, b, n}
    {2, 5},       # B4 {c, j}
    {1, 4, 3},    # B5 {m, p, b}
    {1, 2, 3, 5}, # B6 {m, c, b, j}
    {2, 3, 5},    # B7 {c, b, j}
    {3, 2}        # B8 {b, c}
]


L, C = A_priori(transactions, s=3, max_k=3)

C1: 870 - [frozenset({0}), frozenset({1}), frozenset({2}), frozenset({3}), frozenset({4})]..
L1: 4 - [frozenset({1}), frozenset({2}), frozenset({3}), frozenset({5})]..
C2 candidates: 6 - [frozenset({1, 2}), frozenset({1, 3}), frozenset({1, 5}), frozenset({2, 3}), frozenset({2, 5})]..


100%|██████████| 8/8 [00:00<00:00, 106861.25it/s]


C2 counts: 6 - [frozenset({1, 2}), frozenset({1, 3}), frozenset({1, 5}), frozenset({2, 3}), frozenset({2, 5})]..
L2: 4 - {frozenset({1, 2}): 3, frozenset({1, 3}): 4, frozenset({2, 3}): 5, frozenset({2, 5}): 3}
C3 candidates: 1 - [frozenset({1, 2, 3})]..


100%|██████████| 8/8 [00:00<00:00, 294337.12it/s]

C3 counts: 1 - [frozenset({1, 2, 3})]..
L3: 1 - {frozenset({1, 2, 3}): 3}


In [129]:

import itertools
from typing import List, Dict, FrozenSet

def find_association_rules(L:List[Dict[FrozenSet[int], int]], c=0.5):
    '''
    L: list of frequent itemsets (L1, L2, ... Lk), where Lk is a dict of itemsets and their counts
    '''
    
    # get all frequent itemsets
    frequent_itemsets = {}
    for level in L:
        frequent_itemsets.update(level)
    
    # filter out singletons
    frequent_itemsets = list(k for k in frequent_itemsets.keys() if len(k) > 1)
    
    # support counts
    counts = {}
    for level in L:
        counts.update(level)
    
    # generate rules
    rules = {}
    for itemset in frequent_itemsets:
        n = len(itemset)
        for i in range(1, n):
            for subset in map(frozenset, itertools.combinations(itemset, i)):
                consequent = itemset - subset
                
                # evaluate subset -> consequent
                
                if len(consequent) > 0:  # Ensure non-empty consequent
                    conf = counts[itemset] / counts[subset]
                    if conf >= c:
                        rules[(subset, consequent)] = conf
    
    return rules


In [130]:
find_association_rules(L, c=0.75)

{(frozenset({1}), frozenset({3})): 0.8,
 (frozenset({2}), frozenset({3})): 0.8333333333333334,
 (frozenset({3}), frozenset({2})): 0.8333333333333334,
 (frozenset({5}), frozenset({2})): 0.75,
 (frozenset({1, 2}), frozenset({3})): 1.0,
 (frozenset({1, 3}), frozenset({2})): 0.75}

# Putting it all together

1. Find frequent itemsets
2. Find association rules

In [131]:
transactions = load_transactions('data/T10I4D100K.dat')

s = 0.01*len(transactions)
print(f"s: {s}")
L, C = A_priori(transactions, s, 3)



s: 1000.0
C1: 870 - [frozenset({0}), frozenset({1}), frozenset({2}), frozenset({3}), frozenset({4})]..
L1: 375 - [frozenset({1}), frozenset({4}), frozenset({5}), frozenset({6}), frozenset({8})]..
C2 candidates: 70125 - [frozenset({1, 4}), frozenset({1, 5}), frozenset({1, 6}), frozenset({8, 1}), frozenset({1, 10})]..


100%|██████████| 100000/100000 [03:13<00:00, 516.39it/s]


C2 counts: 70125 - [frozenset({1, 4}), frozenset({1, 5}), frozenset({1, 6}), frozenset({8, 1}), frozenset({1, 10})]..
L2: 9 - {frozenset({704, 39}): 1107, frozenset({825, 39}): 1187, frozenset({217, 346}): 1336, frozenset({227, 390}): 1049, frozenset({368, 682}): 1193, frozenset({368, 829}): 1194, frozenset({722, 390}): 1042, frozenset({704, 825}): 1102, frozenset({829, 789}): 1194}
C3 candidates: 1 - [frozenset({704, 825, 39})]..


100%|██████████| 100000/100000 [00:00<00:00, 1997401.76it/s]

C3 counts: 1 - [frozenset({704, 825, 39})]..
L3: 1 - {frozenset({704, 825, 39}): 1035}


In [132]:
find_association_rules(L, c=0.5)

{(frozenset({704}), frozenset({39})): 0.617056856187291,
 (frozenset({227}), frozenset({390})): 0.577007700770077,
 (frozenset({704}), frozenset({825})): 0.6142697881828316,
 (frozenset({704}), frozenset({39, 825})): 0.5769230769230769,
 (frozenset({704, 825}), frozenset({39})): 0.9392014519056261,
 (frozenset({39, 704}), frozenset({825})): 0.9349593495934959,
 (frozenset({39, 825}), frozenset({704})): 0.8719460825610783}

# Conclusion
The association rules suggests that if 704 and 825 is bought, then 39 is also bought, with a high condifence of almost 94%. There seem to be a strong correlation between all of these three products, while it seems like 39 and 825 is more often bought without 704 than the others. 

if 704 is bought, then you can be quite confident of the customer also buying 30 or 825 ~60%
if 704 is bought with either 39 or 825, then with a confidence of more than 93% we can say that they'll also buy all three. 


It would be clever to arrange all of these products close to each other in the store.

226 and 390 also seems to have some correlation, yet lower at conf~58%